# TOKENIZATION

In [1]:
import nltk

In [86]:
paragraph = """I have three visions for India. In 3000 years of our history, people from all over 
               the world have come and invaded us, captured our lands, conquered our minds. 
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours. 
               Yet we have not done this to any other nation. We have not conquered anyone. 
               We have not grabbed their land, their culture, 
               their history and tried to enforce our way of life on them. 
               Why? Because we respect the freedom of others.That is why my 
               first vision is that of freedom. I believe that India got its first vision of 
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India 
               stands up to the world, no one will respect us. Only strength respects strength. We must be 
               strong not only as a military power but also as an economic power. Both must go hand-in-hand. 
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of 
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life. 
               I see four milestones in my career"""

In [87]:
sentences=nltk.sent_tokenize(paragraph)

In [57]:
words=nltk.word_tokenize(paragraph)

# STEMMING

In [58]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [59]:
new_obj=PorterStemmer()
for i in range(len(sentences)):
    stem_list=[]
    words = nltk.word_tokenize(sentences[i])
    for word in words:
        if word not in set(stopwords.words('english')):
            stem_word = new_obj.stem(word)
            stem_list.append(stem_word)
    sentences[i]=' '.join(stem_list) 
    

In [45]:
# or just

# words = [new_obj.stem(word) for word in words if word not in set(stopwords.words('english'))]
# words=' '.join(words)

# Lemmatization

In [65]:
from nltk.stem import WordNetLemmatizer

In [74]:
new_obj=WordNetLemmatizer()
for i in range(len(sentences)):
    stem_list=[]
    words = nltk.word_tokenize(sentences[i])
    for word in words:
        if word not in set(stopwords.words('english')):
            stem_word = new_obj.lemmatize(word)
            stem_list.append(stem_word)
    sentences[i]=' '.join(stem_list)

# BAG OF WORDS

In [77]:
import re

In [88]:
ps = PorterStemmer()
wordnet=WordNetLemmatizer()
sentences = nltk.sent_tokenize(paragraph)
corpus = []
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
count_obj=CountVectorizer(max_features=1500)
X_bow=count_obj.fit_transform(corpus).toarray()

In [95]:
X_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# TF-IDF vectorizer

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_obj=TfidfVectorizer()
X_tfidf=tfidf_obj.fit_transform(corpus).toarray()

In [99]:
X_tfidf.shape

(31, 113)

# Spam detection in sms

# a.using lemmatizer and bag of words representation

In [155]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

table=pd.read_csv(r'SMSSpamCollection',sep='\t',names=["label", "message"])

# data cleaning and preprocessing

corpus=[]
lemm_obj=WordNetLemmatizer()

for i in range(len(table)):
    review = re.sub('[^a-zA-Z]', ' ', table['message'][i])
    review = review.lower()
    review = review.split()
    
    lem_rev = [lemm_obj.lemmatize(word) for word in review if not word in stopwords.words('english')]
    lem_rev = ' '.join(lem_rev)
    corpus.append(lem_rev)
    
# bag of words representation

from sklearn.feature_extraction.text import CountVectorizer
sms_bow=CountVectorizer(max_features=5000)
X_bow=sms_bow.fit_transform(corpus).toarray()

y=pd.get_dummies(table['label'])
y=y.iloc[:,1].values

#splitting the data

from sklearn.model_selection import train_test_split
X_tr , X_te , y_tr , y_te = train_test_split(X_bow,y,test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_tr,y_tr)

y_pred=model.predict(X_te)


In [157]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_te, y_pred)
print(con)

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_te, y_pred)
print(accuracy)

[[944  11]
 [  9 151]]
0.9820627802690582


# b. lemmatizer and tfidf representation

In [158]:
# data cleaning and preprocessing

corpus=[]
lemm_obj=WordNetLemmatizer()

for i in range(len(table)):
    review = re.sub('[^a-zA-Z]', ' ', table['message'][i])
    review = review.lower()
    review = review.split()
    
    lem_rev = [lemm_obj.lemmatize(word) for word in review if not word in stopwords.words('english')]
    lem_rev = ' '.join(lem_rev)
    corpus.append(lem_rev)
    
# tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
sms_tfidf=TfidfVectorizer(max_features=5000)
X_tfidf=sms_tfidf.fit_transform(corpus).toarray()

y=pd.get_dummies(table['label'])
y=y.iloc[:,1].values

#splitting the data

from sklearn.model_selection import train_test_split
X_tr , X_te , y_tr , y_te = train_test_split(X_tfidf,y,test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_tr,y_tr)

y_pred=model.predict(X_te)


In [159]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_te, y_pred)
print(con)

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_te, y_pred)
print(accuracy)

[[955   0]
 [ 26 134]]
0.9766816143497757


# c.using stemming and BOW representation

In [162]:
# data cleaning and preprocessing

corpus=[]
stem_obj=PorterStemmer()

for i in range(len(table)):
    review = re.sub('[^a-zA-Z]', ' ', table['message'][i])
    review = review.lower()
    review = review.split()
    
    stem_rev = [stem_obj.stem(word) for word in review if not word in stopwords.words('english')]
    stem_rev = ' '.join(stem_rev)
    corpus.append(stem_rev)
    
# bag of words representation

from sklearn.feature_extraction.text import CountVectorizer
sms_bow=CountVectorizer(max_features=5000)
X_bow=sms_bow.fit_transform(corpus).toarray()

y=pd.get_dummies(table['label'])
y=y.iloc[:,1].values

#splitting the data

from sklearn.model_selection import train_test_split
X_tr , X_te , y_tr , y_te = train_test_split(X_bow,y,test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_tr,y_tr)

y_pred=model.predict(X_te)


In [163]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_te, y_pred)
print(con)

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_te, y_pred)
print(accuracy)

[[946   9]
 [  8 152]]
0.9847533632286996


# d.using stemming and tfidf representation

In [170]:
# data cleaning and preprocessing

corpus=[]
stem_obj=PorterStemmer()

for i in range(len(table)):
    review = re.sub('[^a-zA-Z]', ' ', table['message'][i])
    review = review.lower()
    review = review.split()
    
    stem_rev = [stem_obj.stem(word) for word in review if not word in stopwords.words('english')]
    stem_rev = ' '.join(stem_rev)
    corpus.append(stem_rev)
    
# tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
sms_tfidf=TfidfVectorizer(max_features=5000)
X_tfidf=sms_tfidf.fit_transform(corpus).toarray()

y=pd.get_dummies(table['label'])
y=y.iloc[:,1].values

#splitting the data

from sklearn.model_selection import train_test_split
X_tr , X_te , y_tr , y_te = train_test_split(X_tfidf,y,test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_tr,y_tr)

y_pred=model.predict(X_te)


In [171]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_te, y_pred)
print(con)

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_te, y_pred)
print(accuracy)

[[955   0]
 [ 29 131]]
0.9739910313901345


# WORD2VEC

In [172]:
import nltk

from gensim.models import Word2Vec
from nltk.corpus import stopwords

import re

C:\Users\SHRAVAN G H\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [175]:
paragraph = """I have three visions for India. In 3000 years of our history, people from all over 
               the world have come and invaded us, captured our lands, conquered our minds. 
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours. 
               Yet we have not done this to any other nation. We have not conquered anyone. 
               We have not grabbed their land, their culture, 
               their history and tried to enforce our way of life on them. 
               Why? Because we respect the freedom of others.That is why my 
               first vision is that of freedom. I believe that India got its first vision of 
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India 
               stands up to the world, no one will respect us. Only strength respects strength. We must be 
               strong not only as a military power but also as an economic power. Both must go hand-in-hand. 
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of 
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life. 
               I see four milestones in my career"""



# Preprocessing the data
text = re.sub(r'\[[0-9]*\]',' ',paragraph)
text = re.sub(r'\s+',' ',text)
text = text.lower()
text = re.sub(r'\d',' ',text)
text = re.sub(r'\s+',' ',text)

# Preparing the dataset
sentences = nltk.sent_tokenize(text)

sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    
    
# Training the Word2Vec model
model = Word2Vec(sentences, min_count=1)



In [185]:
words = model.wv.key_to_index.keys()
print(words)
print(len(words))

dict_keys(['.', ',', 'india', 'vision', 'must', 'nation', 'world', 'us', 'three', 'freedom', 'respect', 'see', 'first', 'power', 'yet', '’', 'strength', 'worked', '?', 'life', 'believe', 'dr.', 'great', 'minds', 'one', 'years', 'history', 'developed', 'conquered', 'protect', 'others.that', 'war', 'nurture', 'independence', 'build', 'free', 'tried', 'got', 'started', 'way', 'enforce', 'took', 'culture', 'turks', 'visions', 'people', 'come', 'invaded', 'captured', 'lands', 'alexander', 'onwards', 'greeks', 'moguls', 'land', 'portuguese', 'british', 'french', 'dutch', 'came', 'looted', 'done', 'anyone', 'grabbed', 'second', 'career', 'development', 'respects', 'military', 'also', 'economic', 'go', 'hand-in-hand', 'good', 'fortune', 'vikram', 'sarabhai', 'dept', 'space', 'professor', 'satish', 'dhawan', 'succeeded', 'brahm', 'prakash', 'father', 'nuclear', 'material', 'lucky', 'closely', 'consider', 'opportunity', 'four', 'strong', 'stands', 'fifty', 'unless', 'developing', 'milestones', '

In [176]:
# Finding Word Vectors
vector = model.wv['war']
vector

array([-0.00219905, -0.00970885,  0.00929075,  0.00203636, -0.00116388,
       -0.00551955, -0.0085126 , -0.00989383,  0.00894091, -0.00250522,
        0.00459427, -0.00452481,  0.00995189,  0.00366171,  0.00103129,
       -0.00403834,  0.00122027, -0.00265451,  0.00735284,  0.00447542,
        0.00099955,  0.0034782 ,  0.00372712, -0.00680036,  0.00893242,
        0.00173499, -0.00579935,  0.00866838, -0.00129286,  0.00818304,
       -0.0014927 ,  0.00698649,  0.00273452, -0.00436226, -0.00374683,
        0.00919046,  0.00159645, -0.00599784,  0.00034776, -0.00195135,
        0.00159242, -0.00771525,  0.00738298,  0.00131083,  0.00787099,
        0.00445568, -0.00439675,  0.00376054, -0.0006357 , -0.00984484,
        0.00825004,  0.00964326,  0.00965426, -0.00379659, -0.00844202,
        0.00483581, -0.00765107,  0.00853567,  0.00275977,  0.00560496,
        0.00611362,  0.00046455, -0.00209463,  0.000778  ,  0.00983559,
       -0.00711718, -0.00155744, -0.00235984,  0.00487084,  0.00

In [187]:
similar=model.wv.most_similar('professor')
similar

[('consider', 0.24767707288265228),
 ('looted', 0.217461958527565),
 ('’', 0.17328032851219177),
 ('invaded', 0.16238373517990112),
 ('war', 0.15752418339252472),
 ('among', 0.1556834876537323),
 ('opportunity', 0.15104511380195618),
 ('respects', 0.147064670920372),
 ('moguls', 0.13638530671596527),
 ('brahm', 0.13400928676128387)]

# Word Embedding Techniques using Embedding Layer in Keras

In [21]:
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [11]:
### sentences
sent=[  'the glass of milk',
     'the glass of juice',
     'the cup of tea',
    'I am a good boy',
     'I am a good developer',
     'understand the meaning of words',
     'your videos are good',]

In [12]:
sent

['the glass of milk',
 'the glass of juice',
 'the cup of tea',
 'I am a good boy',
 'I am a good developer',
 'understand the meaning of words',
 'your videos are good']

In [13]:
### Vocabulary size
voc_size=10000

In [26]:
word_len=5000
one_hot_list=[]
for sentence in sent:
    one_hot_list.append(one_hot(sentence,n=word_len))
print(one_hot_list)    

[[4260, 2529, 3285, 1171], [4260, 2529, 3285, 703], [4260, 3547, 3285, 536], [1826, 3616, 3502, 3719, 721], [1826, 3616, 3502, 3719, 4302], [2843, 4260, 2893, 3285, 945], [912, 2579, 460, 3719]]


In [27]:
pad_len=10
padded_sentences=pad_sequences(one_hot_list,padding='pre',maxlen=pad_len)
print(padded_sentences)

[[   0    0    0    0    0    0 4260 2529 3285 1171]
 [   0    0    0    0    0    0 4260 2529 3285  703]
 [   0    0    0    0    0    0 4260 3547 3285  536]
 [   0    0    0    0    0 1826 3616 3502 3719  721]
 [   0    0    0    0    0 1826 3616 3502 3719 4302]
 [   0    0    0    0    0 2843 4260 2893 3285  945]
 [   0    0    0    0    0    0  912 2579  460 3719]]


In [28]:
model=Sequential()
model.add(Embedding(input_dim=word_len, output_dim=32, input_length=pad_len))
model.compile(optimizer='Adam', loss='MeanSquaredError')

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 10, 32)            160000    
                                                                 
Total params: 160,000
Trainable params: 160,000
Non-trainable params: 0
_________________________________________________________________


In [30]:
print(model.predict(padded_sentences))

[[[ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  [ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  [ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  ...
  [ 0.04406584 -0.03303982 -0.02856502 ...  0.01554526 -0.03949798
   -0.02482674]
  [ 0.00212437 -0.00632347 -0.01938267 ...  0.00989449  0.01241541
    0.010529  ]
  [-0.04738373 -0.03941305  0.03630864 ...  0.00076034  0.03183391
   -0.01803321]]

 [[ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  [ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  [ 0.02154745  0.02888334  0.01670618 ... -0.04185892 -0.01999984
    0.00729573]
  ...
  [ 0.04406584 -0.03303982 -0.02856502 ...  0.01554526 -0.03949798
   -0.02482674]
  [ 0.00212437 -0.00632347 -0.01938267 ...  0.00989449  0.01241541
    0.010529  ]
  [-0.02059616  0.00935761 -0.02212101 ... -0.02731514 -0.01669952
   -0.

In [32]:
padded_sentences[0]

array([   0,    0,    0,    0,    0,    0, 4260, 2529, 3285, 1171])

In [33]:
print(model.predict(padded_sentences)[0])


[[ 0.02154745  0.02888334  0.01670618 -0.03140924 -0.02493899 -0.04150356
   0.00168527  0.02292116  0.02051813  0.01849495 -0.00530056  0.04585899
   0.02039954  0.02585718 -0.00879302  0.02212801 -0.00710664 -0.03175303
   0.04641801 -0.04396261  0.02155112  0.00118432  0.04666677 -0.02996508
  -0.03453206 -0.01944637  0.02987108  0.03403791  0.04143305 -0.04185892
  -0.01999984  0.00729573]
 [ 0.02154745  0.02888334  0.01670618 -0.03140924 -0.02493899 -0.04150356
   0.00168527  0.02292116  0.02051813  0.01849495 -0.00530056  0.04585899
   0.02039954  0.02585718 -0.00879302  0.02212801 -0.00710664 -0.03175303
   0.04641801 -0.04396261  0.02155112  0.00118432  0.04666677 -0.02996508
  -0.03453206 -0.01944637  0.02987108  0.03403791  0.04143305 -0.04185892
  -0.01999984  0.00729573]
 [ 0.02154745  0.02888334  0.01670618 -0.03140924 -0.02493899 -0.04150356
   0.00168527  0.02292116  0.02051813  0.01849495 -0.00530056  0.04585899
   0.02039954  0.02585718 -0.00879302  0.02212801 -0.00710

In [ ]:
#optimizers

# class Adadelta: Optimizer that implements the Adadelta algorithm.

# class Adagrad: Optimizer that implements the Adagrad algorithm.

# class Adam: Optimizer that implements the Adam algorithm.

# class Adamax: Optimizer that implements the Adamax algorithm.

# class Ftrl: Optimizer that implements the FTRL algorithm.

# class Nadam: Optimizer that implements the NAdam algorithm.

# class Optimizer: Base class for Keras optimizers.

# class RMSprop: Optimizer that implements the RMSprop algorithm.

# class SGD: Gradient descent (with momentum) optimizer.


#losses

# Classes

# class BinaryCrossentropy: Computes the cross-entropy loss between true labels and predicted labels.

# class BinaryFocalCrossentropy: Computes the focal cross-entropy loss between true labels and predictions.

# class CategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

# class CategoricalHinge: Computes the categorical hinge loss between y_true and y_pred.

# class CosineSimilarity: Computes the cosine similarity between labels and predictions.

# class Hinge: Computes the hinge loss between y_true and y_pred.

# class Huber: Computes the Huber loss between y_true and y_pred.

# class KLDivergence: Computes Kullback-Leibler divergence loss between y_true and y_pred.

# class LogCosh: Computes the logarithm of the hyperbolic cosine of the prediction error.

# class Loss: Loss base class.

# class MeanAbsoluteError: Computes the mean of absolute difference between labels and predictions.

# class MeanAbsolutePercentageError: Computes the mean absolute percentage error between y_true and y_pred.

# class MeanSquaredError: Computes the mean of squares of errors between labels and predictions.

# class MeanSquaredLogarithmicError: Computes the mean squared logarithmic error between y_true and y_pred.

# class Poisson: Computes the Poisson loss between y_true and y_pred.

# class Reduction: Types of loss reduction.

# class SparseCategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

# class SquaredHinge: Computes the squared hinge loss between y_true and y_pred.


# lstm_imdb

In [1]:
import numpy
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [2]:
from keras.datasets import imdb

In [3]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

17473536/17464789 [==============================] - 5s 0us/step


In [4]:
print(X_train[1])
print(type(X_train[1]))
print(len(X_train[1]))

[1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2637, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 2, 5, 2, 656, 245, 2350, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]
<class 'list'>
189


In [5]:
X_train.shape

(25000,)

In [6]:
max(numpy.max(X_test))

4998

In [7]:
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(25000, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [8]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 600, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

# Final evaluation of the model

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Epoch 1/10
391/391 [==============================] - 451s 1s/step - loss: 0.4995 - accuracy: 0.7567
Epoch 2/10
391/391 [==============================] - 443s 1s/step - loss: 0.3288 - accuracy: 0.8651
Epoch 3/10
391/391 [==============================] - 454s 1s/step - loss: 0.2529 - accuracy: 0.9010
Epoch 4/10
391/391 [==============================] - 464s 1s/step - loss: 0.2275 - accuracy: 0.9113
Epoch 5/10
193/391 [=============>................] - ETA: 4:11 - loss: 0.1960 - accuracy: 0.9271

KeyboardInterrupt: 